<a href="https://colab.research.google.com/github/Guidong99/statistic-optimization/blob/main/NEB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title
from numpy import transpose as t
from numpy import array as array
import numpy as np
import pandas as pd
from scipy.optimize import minimize,Bounds,LinearConstraint
from scipy.special import gamma
from math import exp,pi,log
import matplotlib.pyplot as plt
from scipy.stats import norm
def gaussian(x,mu,sigma2):
  return 1/(2*pi*sigma2)*exp(-(x-mu)**2/2*sigma2)
def chi_square(x,k):
  return (x**(k/2-1)*exp(-x/2))/(2**(k/2)*gamma(k/2)) if x>0 else 0
def generate_C(x,s2,a,b):
  m,K,S=len(x),len(a),len(b)
  C=[]
  c=[[0]*S for i in range(K)]
  for r in range(m):
    for i in range(K):
      for j in range(S):
        c[i][j]=gaussian(x[r],a[i],b[j])*chi_square((n-1)*s2[r]/b[j],n-1)
    C.append(array(c))
  return C



#NEB(Nonelinear Empirical Bayes) class
### 1.arguments:
x
s2
a
b
### 2.functions:
objection function

gradient

hessian



In [ ]:
#@title
class NEB():
  def __init__(self,a,b,C,n):
    # self.x=x
    # self.s2=s2
    self.a=a
    self.b=b
    self.C=C
    self.n=n

  def obj(self):
    k=len(self.a)
    C=self.C
    def obj(u):
      res=0
      for r in range(len(C)):
        res=res*t(u[:k])@C[r]@u[k:]
      return -log(res)
    return obj
  
  def gradient(self):
    C=self.C
    K=len(self.a)
    def l_gradient(g,h,r,i,tp):
      if tp=='g':
        res=(C[r][i]@h)/(t(g)@C[r]@h)
      if tp=='h':
        res=(t(g)@C[r][:,i])/(t(g)@C[r]@h)
      return res
    def gradient(u):
      g,h=u[:K],u[K:]
      res=[]
      for i in range(len(u)):
        if i<K:
          s=sum(l_gradient(g,h,r,i,'g') for r in range(len(C)))
        else:
          s=sum(l_gradient(g,h,r,i-K,'h') for r in range(len(C)))
        res.append(s)
      return res
    return gradient

  def Hess(self):
    K=len(self.a)
    C=self.C
    def Hess(u):
      T=len(u)
      hess=[[0]*T for i in range(T)]
      g,h=u[:K],u[K:]
      def H11(i,j):
        return -sum((t(h)@C[r][i])*(C[r][j]@h)/(t(g)@C[r]@h)**2 for r in range(len(C)))
      def H22(i,j):
        return -sum(t(g)@C[r][:,i]*t(C[r][:,j])@g/(t(g)@C[r]@h)**2 for r in range(len(C)))
      def H12(i,j):
        return sum(C[r][i][j]/(t(g)@C[r]@h)-t(g)@C[r][:,j]*C[r][i]@h/(t(g)@C[r]@h)**2 for r in range(len(C)))
      def H21(i,j):
        return sum(C[r][j][i]/(t(g)@C[r]@h)-t(g)@C[r][:,i]*C[r][j]@h/(t(g)@C[r]@h)**2 for r in range(len(C)))
      for i in range(T):
        for j in range(T):
          if i<K:
            if j<K: hess[i][j]=H11(i,j)
            else: hess[i][j]=H12(i,j-K)
          else:
            if j<K: hess[i][j]=H21(i-K,j)
            else: hess[i][j]=H22(i-K,j-K)
      return hess
    return Hess
  def constraints(self):
    constraint=[]
    K,S=len(self.a),len(self.b)
    constraint.append([1]*K+[0]*S)
    constraint.append([0]*K+[1]*S)
    bound0,bound1=[0]*(K+S),[np.inf]*(K+S)
    return constraint,bound0,bound1


#Example

In [ ]:
np.random.seed(10086)
m=500
mu=[0]*m
pi0=0.5
mu[:round(pi0*m)]=np.random.normal(size=round(pi0*m))
s=[1]*m
n=10
data=np.zeros([m,n]) 
for i in range(m):
  data[i]=np.random.normal(mu[i],s[i],n)
x=data.mean(axis=1)
s2=data.std(axis=1)**2/n
K=20
a=(max(abs(x))/K)*np.linspace(-K/2,K/2,K)
S=10
b=np.linspace(min(s2),max(s2),L)

 
 

s=NEB(a,b,generate_C(x,s2,a,b),n)
Obj=s.obj()
Grad=s.gradient()
Hess=s.Hess()
constraints,bound0,bound1=s.constraints()
x0=[1]*(K+S)

In [ ]:
# pd.DataFrame(generate_C(x,s2,a,b)[0])
len(generate_C(x,s2,a,b))

#Tools Camparing

In [ ]:
# bound1

linear_constraint=LinearConstraint(constraints,[1,1],[1,1])
bounds=Bounds(bound0,bound1)

In [ ]:
res = minimize(Obj, x0, method='trust-constr', jac=Grad, hess=Hess,
               constraints=linear_constraint,
               options={'verbose': 1}, bounds=bounds)

u=res.x

In [ ]:
u[:K]


plt.plot(u[:K])

In [ ]:
s=N